In [1]:
import gzip
import json
import pandas as pd
import numpy as np
import os

In [21]:
def load_data(file_name, head = 1):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
            if (head is not None) and (count > head):
                break
    return data

DIR='./datasets/other_datasets/goodreads_dataset_complete/datas/'

In [28]:
genres = load_data(os.path.join(DIR, 'goodreads_book_genres_initial.json.gz'), head=None)
genres_df=pd.DataFrame(genres)
genres_df.columns

Index(['book_id', 'genres'], dtype='object')

In [40]:
authors = load_data(os.path.join(DIR, 'goodreads_book_authors.json.gz'), head=None)
authors_df=pd.DataFrame(authors)
authors_df.columns

Index(['average_rating', 'author_id', 'text_reviews_count', 'name',
       'ratings_count'],
      dtype='object')

In [11]:
books = load_data(os.path.join(DIR, 'goodreads_books.json.gz'))
books_df_gdr=pd.DataFrame(books)
books_df_gdr.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')

In [4]:
works = load_data(os.path.join(DIR, 'goodreads_book_works.json.gz'))
works_df=pd.DataFrame(works)
works_df.columns

Index(['books_count', 'reviews_count', 'original_publication_month',
       'default_description_language_code', 'text_reviews_count',
       'best_book_id', 'original_publication_year', 'original_title',
       'rating_dist', 'default_chaptering_book_id', 'original_publication_day',
       'original_language_id', 'ratings_count', 'media_type', 'ratings_sum',
       'work_id'],
      dtype='object')

In [33]:
works_df.head()

,books_count,reviews_count,original_publication_month,default_description_language_code,text_reviews_count,best_book_id,original_publication_year,original_title,rating_dist,default_chaptering_book_id,original_publication_day,original_language_id,ratings_count,media_type,ratings_sum,work_id
0,1,6,8,,1,5333265,1984,W. C. Fields: A Life on Film,5:1|4:1|3:1|2:0|1:0|total:3,,,,3,book,12,5400751
1,22,10162,,,741,25717,2001,Good Harbor,5:517|4:1787|3:2763|2:966|1:196|total:6229,,,,6229,book,20150,1323437


In [5]:
reviews = load_data(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'))
reviews_df=pd.DataFrame(reviews)
reviews_df.columns

Index(['user_id', 'book_id', 'review_id', 'rating', 'review_text',
       'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes',
       'n_comments'],
      dtype='object')

In [2]:
columns_books = ['isbn', 'isbn13', 'book_id', 'work_id', 'language_code', 'title', 'authors', 'publisher', 'publication_year', 'average_rating', 'description', 'image_url', 'url']

In [3]:
columns_works = [
    'work_id',                      # ID unique de l'œuvre (clé primaire)
    'best_book_id',                 # ID du livre principal associé
    'original_title',               # titre original
    'original_publication_year',    # année de publication
    'original_publication_month',   # mois de publication
    'original_publication_day',     # jour de publication
    'original_language_id',         # langue originale (code ou ID)
    'books_count',                  # nombre de livres de l'œuvre
    'ratings_count',                # total des votes
    'ratings_sum',                  # somme des notes (utile pour moyenne)
    'text_reviews_count',           # nombre de reviews texte
    'media_type',                   # type de média (ebook, hardcover, etc.)
    'default_description_language_code',  # langue de la description                   # distribution des ratings (JSON)
]


In [4]:
columns_reviews = [
    'review_id',     # ID unique de la review (clé primaire)
    'user_id',       # ID de l'utilisateur
    'book_id',       # ID du livre
    'rating',        # note donnée par l'utilisateur
    'review_text',   # texte de la review (optionnel selon usage)
    'date_added',    # date d'ajout
    'date_updated',  # date de mise à jour
    'read_at',       # date de lecture
    'started_at',    # date de début de lecture
    'n_votes',       # nombre de votes utiles
    'n_comments'     # nombre de commentaires
]


In [15]:
user='postgres'
password='redsql'

In [16]:
import gzip
import ujson as json
from sqlalchemy import create_engine, text
import psycopg2
from psycopg2.extras import execute_values
from pathlib import Path

# --- CONFIG ---
DIR = Path('./datasets/other_datasets/goodreads_dataset_complete/datas/')
DB_PARAMS = {
    "dbname": "goodreads",
    "user": user,
    "password": password,
    "host": "localhost"
}
BATCH_SIZE = 200_000

# Colonnes numériques
NUMERIC_COLUMNS = {
    'books': ['book_id', 'work_id', 'publication_year', 'average_rating'],
    'works': ['work_id', 'best_book_id', 'original_publication_year', 'original_publication_month',
              'original_publication_day', 'original_language_id', 'books_count', 'ratings_count', 'ratings_sum',
              'text_reviews_count'],
    'reviews': ['book_id', 'rating', 'n_votes', 'n_comments']
}

# Colonnes JSON
JSON_COLUMNS = {
    'books': ['authors'],
    'works': ['rating_dist'],
    'reviews': []
}

# Définition des fichiers et colonnes
FILES = {
    "books": {
        "file": DIR / "goodreads_books.json.gz",
        "columns": ['book_id','isbn','isbn13','work_id','language_code','title','authors','publisher',
                    'publication_year','average_rating','description','image_url','url'],
        "primary_key": 'book_id'
    },
    "works": {
        "file": DIR / "goodreads_book_works.json.gz",
        "columns": ['work_id','best_book_id','original_title','original_publication_year',
                    'original_publication_month','original_publication_day','original_language_id','books_count',
                    'ratings_count','ratings_sum','text_reviews_count','media_type',
                    'default_description_language_code','rating_dist'],
        "primary_key": 'work_id'
    },
    "reviews": {
        "file": DIR / "goodreads_reviews_dedup.json.gz",
        "columns": ['review_id', 'user_id', 'book_id', 'rating', 'n_votes', 'n_comments'],
        "primary_key": 'review_id'
    },
}


In [17]:
# --- FONCTIONS ---
def create_table(engine, table_name, columns, primary_key=None):
    """Crée la table avec types adaptés et clé primaire"""
    cols_with_types = []
    for col in columns:
        if col in NUMERIC_COLUMNS.get(table_name, []):
            sql_type = 'BIGINT' if 'id' in col else 'DOUBLE PRECISION' if col=='average_rating' else 'INTEGER'
        elif col in JSON_COLUMNS.get(table_name, []):
            sql_type = 'JSONB'
        else:
            sql_type = 'TEXT'
        if primary_key and col == primary_key:
            sql_type += ' PRIMARY KEY'
        cols_with_types.append(f"{col} {sql_type}")
    cols_sql = ", ".join(cols_with_types)

    with engine.begin() as conn:
        conn.execute(text(f"DROP TABLE IF EXISTS {table_name} CASCADE;"))
        conn.execute(text(f"CREATE TABLE {table_name} ({cols_sql});"))
        print(f"{table_name} recréée ✅")

def write_json_gz_to_postgres_safe(file_path, table_name, columns, primary_key=None, batch_size=BATCH_SIZE):
    """Insertion rapide avec psycopg2 execute_values"""
    engine = create_engine(f"postgresql://{DB_PARAMS['user']}:{DB_PARAMS['password']}@{DB_PARAMS['host']}/{DB_PARAMS['dbname']}")
    create_table(engine, table_name, columns, primary_key)

    conn = psycopg2.connect(**DB_PARAMS)
    cur = conn.cursor()
    batch = []

    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f, 1):
            data = json.loads(line)
            row = []
            for col in columns:
                val = data.get(col)
                # Conversion numérique
                if col in NUMERIC_COLUMNS.get(table_name, []):
                    if val in (None, '', 'None'):
                        val = None
                    else:
                        val = int(float(val))
                # JSON
                if col in JSON_COLUMNS.get(table_name, []):
                    val = json.dumps(val if val else [])
                row.append(val)
            batch.append(row)

            if i % batch_size == 0:
                execute_values(cur,
                               f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s",
                               batch)
                conn.commit()
                print(f"{table_name}: batch {i//batch_size} écrit ({i} lignes)")
                batch = []

        if batch:
            execute_values(cur,
                           f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s",
                           batch)
            conn.commit()
            print(f"{table_name}: dernier batch écrit, total lignes {i}")

    cur.close()
    conn.close()

In [28]:
# --- MAIN ---
file = FILES['reviews']

write_json_gz_to_postgres_safe(file['file'], 'reviews', file['columns'], file.get('primary_key'))

reviews recréée ✅
reviews: batch 1 écrit (200000 lignes)
reviews: batch 2 écrit (400000 lignes)
reviews: batch 3 écrit (600000 lignes)
reviews: batch 4 écrit (800000 lignes)
reviews: batch 5 écrit (1000000 lignes)
reviews: batch 6 écrit (1200000 lignes)
reviews: batch 7 écrit (1400000 lignes)
reviews: batch 8 écrit (1600000 lignes)
reviews: batch 9 écrit (1800000 lignes)
reviews: batch 10 écrit (2000000 lignes)
reviews: batch 11 écrit (2200000 lignes)
reviews: batch 12 écrit (2400000 lignes)
reviews: batch 13 écrit (2600000 lignes)
reviews: batch 14 écrit (2800000 lignes)
reviews: batch 15 écrit (3000000 lignes)
reviews: batch 16 écrit (3200000 lignes)
reviews: batch 17 écrit (3400000 lignes)
reviews: batch 18 écrit (3600000 lignes)
reviews: batch 19 écrit (3800000 lignes)
reviews: batch 20 écrit (4000000 lignes)
reviews: batch 21 écrit (4200000 lignes)
reviews: batch 22 écrit (4400000 lignes)
reviews: batch 23 écrit (4600000 lignes)
reviews: batch 24 écrit (4800000 lignes)
reviews: ba

In [18]:
#pour le faire sur les 3 tables
for table, info in FILES.items():
    print(f"Import {table} depuis {info['file']}...")
    write_json_gz_to_postgres_safe(info['file'], table, info['columns'], info.get('primary_key'))

Import books depuis datasets\other_datasets\goodreads_dataset_complete\datas\goodreads_books.json.gz...
books recréée ✅
books: batch 1 écrit (200000 lignes)
books: batch 2 écrit (400000 lignes)
books: batch 3 écrit (600000 lignes)
books: batch 4 écrit (800000 lignes)
books: batch 5 écrit (1000000 lignes)
books: batch 6 écrit (1200000 lignes)
books: batch 7 écrit (1400000 lignes)
books: batch 8 écrit (1600000 lignes)
books: batch 9 écrit (1800000 lignes)
books: batch 10 écrit (2000000 lignes)
books: batch 11 écrit (2200000 lignes)
books: dernier batch écrit, total lignes 2360655
Import works depuis datasets\other_datasets\goodreads_dataset_complete\datas\goodreads_book_works.json.gz...
works recréée ✅
works: batch 1 écrit (200000 lignes)
works: batch 2 écrit (400000 lignes)
works: batch 3 écrit (600000 lignes)
works: batch 4 écrit (800000 lignes)
works: batch 5 écrit (1000000 lignes)
works: batch 6 écrit (1200000 lignes)
works: batch 7 écrit (1400000 lignes)
works: dernier batch écrit, 

In [20]:
# Créer les index pour accélérer les filtres
engine = create_engine(f"postgresql://{DB_PARAMS['user']}:{DB_PARAMS['password']}@{DB_PARAMS['host']}/{DB_PARAMS['dbname']}")
with engine.connect() as conn:
    conn.execute(text("CREATE INDEX IF NOT EXISTS idx_books_book_id ON books(book_id);"))
    conn.execute(text("CREATE INDEX IF NOT EXISTS idx_works_work_id ON works(work_id);"))
    conn.execute(text("CREATE INDEX IF NOT EXISTS idx_review_book_id ON reviews(book_id);"))
    print("Index créés ✅")

Index créés ✅


In [22]:
reviews = load_data(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'), head=3000000)
reviews_df=pd.DataFrame(reviews)

In [23]:
reviews_df = reviews_df[reviews_df['rating'] > 0]

In [24]:
reviews_df.shape

(2898824, 11)

In [25]:
book_ids = reviews_df['book_id'].dropna().astype(int).unique().tolist()

In [61]:
from sqlalchemy import text

# Charger les books correspondants
query_books = text("""
SELECT * FROM books
WHERE book_id = ANY(:book_ids)
""")

with engine.connect() as conn:
    books_df = pd.read_sql_query(query_books, conn, params={"book_ids": book_ids})

# Charger les works correspondants
books_df['work_id'] = books_df['work_id'].dropna().astype(int)
work_ids = books_df['work_id'].unique().tolist()
query_works = text("""
SELECT * FROM works
WHERE work_id = ANY(:work_ids)
""")

with engine.connect() as conn:
    works_df = pd.read_sql_query(query_works, conn, params={"work_ids": work_ids})


In [62]:
books_df['work_id'] = books_df['work_id'].apply(
    lambda x: int(x) if pd.notna(x) else pd.NA
)

In [63]:
books_df['publication_year'] = books_df['publication_year'].apply(
    lambda x: int(x) if pd.notna(x) else pd.NA
)

In [64]:
books_df.isnull().sum()

book_id                  0
isbn                     0
isbn13                   0
work_id                 45
language_code            0
title                    0
authors                  0
publisher                0
publication_year    167236
average_rating          45
description              0
image_url                0
url                      0
dtype: int64

In [65]:
books_df.head()

,book_id,isbn,isbn13,work_id,language_code,title,authors,publisher,publication_year,average_rating,description,image_url,url
0,1,0439785960,9780439785969,41335427,eng,Harry Potter and the Half-Blood Prince (Harry ...,"[{'role': '', 'author_id': '1077326'}, {'role'...",Scholastic Inc.,2006,4.0,The war against Voldemort is not going well: e...,https://images.gr-assets.com/books/1361039191m...,https://www.goodreads.com/book/show/1.Harry_Po...
1,2,0439358078,9780439358071,2809203,eng,Harry Potter and the Order of the Phoenix (Har...,"[{'role': '', 'author_id': '1077326'}, {'role'...",Scholastic Inc.,2004,4.0,Harry Potter is due to start his fifth year at...,https://images.gr-assets.com/books/1507396732m...,https://www.goodreads.com/book/show/2.Harry_Po...
2,3,0439554934,9780439554930,4640799,eng,Harry Potter and the Sorcerer's Stone (Harry P...,"[{'role': '', 'author_id': '1077326'}, {'role'...",Scholastic Inc,1997,4.0,Harry Potter's life is miserable. His parents ...,https://images.gr-assets.com/books/1474154022m...,https://www.goodreads.com/book/show/3.Harry_Po...
3,4,0439554896,9780439554893,6231171,eng,Harry Potter and the Chamber of Secrets (Harry...,"[{'role': '', 'author_id': '1077326'}]",Scholastic,2003,4.0,The Dursleys were so mean and hideous that sum...,https://images.gr-assets.com/books/1474154604m...,https://www.goodreads.com/book/show/4.Harry_Po...
4,5,043965548X,9780439655484,2402163,eng,Harry Potter and the Prisoner of Azkaban (Harr...,"[{'role': '', 'author_id': '1077326'}, {'role'...",Scholastic Inc.,2004,4.0,Harry Potter's third year at Hogwarts is full ...,https://images.gr-assets.com/books/1499277281m...,https://www.goodreads.com/book/show/5.Harry_Po...


In [49]:
works_df.head()

,work_id,best_book_id,original_title,original_publication_year,original_publication_month,original_publication_day,original_language_id,books_count,ratings_count,ratings_sum,text_reviews_count,media_type,default_description_language_code,rating_dist
0,1323437,25717,Good Harbor,2001.0,NaN,NaN,None,22,6229,20150,741,book,,5:517|4:1787|3:2763|2:966|1:196|total:6229
1,8948723,7327624,,1987.0,NaN,NaN,None,2,141,568,7,book,,5:49|4:58|3:26|2:5|1:3|total:141
2,6243154,6066819,Best Friends Forever,2009.0,7.0,14.0,None,38,53273,185670,3504,book,,5:9152|4:16855|3:19507|2:6210|1:1549|total:53273
3,368291,378460,The Wanting of Levine,NaN,NaN,NaN,None,2,13,57,4,,,5:7|4:4|3:2|2:0|1:0|total:13
4,701117,18948499,All's Fairy in Love and War (Avalon: Quest for...,2003.0,11.0,1.0,None,11,779,3288,18,book,,5:382|4:226|3:135|2:33|1:3|total:779


In [68]:
works_df['original_publication_year'] = works_df['original_publication_year'].apply(
    lambda x: int(x) if pd.notna(x) else pd.NA
)
works_df['original_publication_month'] = works_df['original_publication_month'].apply(
    lambda x: int(x) if pd.notna(x) else pd.NA
)
works_df['original_publication_day'] = works_df['original_publication_day'].apply(
    lambda x: int(x) if pd.notna(x) else pd.NA
)

In [70]:
import pandas as pd
import numpy as np

def make_date_safe(row):
    # Vérifier que year est valide
    year = row['original_publication_year']
    if pd.isna(year) or not (1000 <= int(year) <= 2100):
        return pd.NaT
    
    month = int(row['original_publication_month']) if pd.notna(row['original_publication_month']) else 1
    day = int(row['original_publication_day']) if pd.notna(row['original_publication_day']) else 1

    # Si month ou day invalide, corriger
    month = min(max(month,1),12)
    day = min(max(day,1),31)

    return pd.Timestamp(year=int(year), month=month, day=day)

works_df['original_publication_date'] = works_df.apply(make_date_safe, axis=1)

# Vérifier
works_df[['original_publication_year', 'original_publication_month', 'original_publication_day', 'original_publication_date']]


,original_publication_year,original_publication_month,original_publication_day,original_publication_date
0,2001,<NA>,<NA>,2001-01-01 00:00:00
1,1987,<NA>,<NA>,1987-01-01 00:00:00
2,2009,7,14,2009-07-14 00:00:00
3,<NA>,<NA>,<NA>,NaT
4,2003,11,1,2003-11-01 00:00:00
...,...,...,...,...
550403,2013,2,13,2013-02-13 00:00:00
550404,2014,10,15,2014-10-15 00:00:00
550405,2009,<NA>,<NA>,2009-01-01 00:00:00
550406,2007,12,4,2007-12-04 00:00:00


In [76]:
reviews_df.to_csv('./reco_books/data/raw/goodreads/reviews.csv', index=False)

In [77]:
books_df.to_csv('./reco_books/data/raw/goodreads/books.csv', index=False)

In [ ]:
works_df.to_csv('./reco_books/data/raw/goodreads/works.csv', index=False)

: 

In [ ]:
authors_df.to_csv('./reco_books/data/raw/goodreads/authors.csv', index=False)

In [ ]:
genres_df.to_csv('./reco_books/data/raw/goodreads/categories.csv', index=False)

In [ ]:
reviews_df = pd.read_csv('./reco_books/data/raw/goodreads/reviews.csv')
works_df=pd.read_csv('./reco_books/data/raw/goodreads/works.csv')
books_df=pd.read_csv('./reco_books/data/raw/goodreads/books.csv')
authors_df=pd.read_csv('./reco_books/data/raw/goodreads/authors.csv')
genres_df = pd.read_csv('./reco_books/data/raw/goodreads/categories.csv')

In [49]:
books_df['book_id'].duplicated().sum()

0

In [50]:
books_df = books_df.drop_duplicates(subset='book_id')
books_en = books_df[books_df['language_code'].isin(['eng','en-US','en-GB','en-CA'])].copy()
books_int = books_df[~books_df['book_id'].isin(books_en['book_id'])].copy()
books_int['language_code'].unique()

array([nan, 'spa', 'fre', 'por', 'grc', 'enm', 'jpn', 'ger', 'ara', 'rus',
       'lat', 'zho', 'ita', 'mul', 'nl', 'swe', 'nor', 'ind', 'ale',
       'tur', 'per', 'ira', 'pes', 'pol', 'vie', 'frs', 'gre', 'fin',
       'myn', 'elx', 'srp', 'lit', 'tlh', 'afr', 'cze', 'raj', 'frm',
       'aus', 'wak', 'hye', 'dan', 'nav', 'ben', 'cat', 'msa', 'hin',
       'sco', 'rum', '--', 'abk', 'heb', 'fil', 'eus', 'tam', 'guj',
       'slo', 'wel', 'nob', 'bul', 'scr', 'nno', 'tgl', 'hun', 'nep',
       'gmh', 'urd', 'mal', 'est', 'tha', 'dut', 'mar', 'nld', 'isl',
       'fao', 'myv', 'slv', 'bos', 'epo', 'kor', 'sin', 'ang', 'kan',
       'lav', 'dum', 'ukr', 'mon', 'sqi', 'sun', 'glg', 'egy', 'und',
       'rup', 'ady', 'amh', 'arg', 'en', 'kir', 'mkd', 'tel', 'kat',
       'aze', 'vls', 'es-MX', 'dua', 'mlt', 'pt-BR', 'inh', 'ava', 'snd',
       'iba', 'tut', 'kur', 'sla', 'sux', 'arw', 'nub', 'phi', 'mus',
       'gle', 'bel', 'nds'], dtype=object)

In [52]:
books_int['language_code'].unique()

array([nan, 'spa', 'fre', 'por', 'grc', 'enm', 'jpn', 'ger', 'ara', 'rus',
       'lat', 'zho', 'ita', 'mul', 'nl', 'swe', 'nor', 'ind', 'ale',
       'tur', 'per', 'ira', 'pes', 'pol', 'vie', 'frs', 'gre', 'fin',
       'myn', 'elx', 'srp', 'lit', 'tlh', 'afr', 'cze', 'raj', 'frm',
       'aus', 'wak', 'hye', 'dan', 'nav', 'ben', 'cat', 'msa', 'hin',
       'sco', 'rum', '--', 'abk', 'heb', 'fil', 'eus', 'tam', 'guj',
       'slo', 'wel', 'nob', 'bul', 'scr', 'nno', 'tgl', 'hun', 'nep',
       'gmh', 'urd', 'mal', 'est', 'tha', 'dut', 'mar', 'nld', 'isl',
       'fao', 'myv', 'slv', 'bos', 'epo', 'kor', 'sin', 'ang', 'kan',
       'lav', 'dum', 'ukr', 'mon', 'sqi', 'sun', 'glg', 'egy', 'und',
       'rup', 'ady', 'amh', 'arg', 'en', 'kir', 'mkd', 'tel', 'kat',
       'aze', 'vls', 'es-MX', 'dua', 'mlt', 'pt-BR', 'inh', 'ava', 'snd',
       'iba', 'tut', 'kur', 'sla', 'sux', 'arw', 'nub', 'phi', 'mus',
       'gle', 'bel', 'nds'], dtype=object)

In [46]:
books_en = books_en.dropna(subset=['description', 'title'])
books_en.shape

(350324, 13)

## Tests avec SVD

In [6]:
from datetime import datetime
import os
import pandas as pd
import numpy as np
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import SVD, SVDpp, NMF
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from collections import defaultdict

In [7]:
# Set Rating Scale
reader = Reader(rating_scale=(1, 5))

# Load data with rating scale
data = Dataset.load_from_df(reviews_df[['user_id', 'book_id', 'rating']], reader)

In [8]:
from surprise import Dataset, SVD
from surprise.model_selection import GridSearchCV

# Search space réduit
param_grid_coarse = {
    'n_factors': [50, 100, 150],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.01, 0.02, 0.05],
    'n_epochs': [20]  # on fixe un nombre d'époques raisonnable pour aller vite
}

In [9]:
gs_coarse = GridSearchCV(
    SVD,
    param_grid_coarse,
    measures=['rmse'],
    cv=3,
    n_jobs=-1,
    joblib_verbose=1
)

In [10]:
gs_coarse.fit(data)

print("Coarse search best RMSE:", gs_coarse.best_score['rmse'])
print("Coarse search best params:", gs_coarse.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  2.1min


Coarse search best RMSE: 0.9065774142452966
Coarse search best params: {'n_factors': 50, 'lr_all': 0.005, 'reg_all': 0.05, 'n_epochs': 20}


[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  6.0min finished


In [11]:
best_params = gs_coarse.best_params['rmse']

param_grid_fine = {
    'n_factors': [best_params['n_factors']-25, best_params['n_factors'], best_params['n_factors']+25],
    'lr_all': [best_params['lr_all']*0.5, best_params['lr_all'], best_params['lr_all']*1.5],
    'reg_all': [best_params['reg_all']*0.5, best_params['reg_all'], best_params['reg_all']*1.5],
    'n_epochs': [best_params['n_epochs'], best_params['n_epochs']+10, best_params['n_epochs']+20]
}

gs_fine = GridSearchCV(
    SVD,
    param_grid_fine,
    measures=['rmse'],
    cv=3,
    n_jobs=-1,
    joblib_verbose=1
)

In [12]:
gs_fine.fit(data)

print("Fine search best RMSE:", gs_fine.best_score['rmse'])
print("Fine search best params:", gs_fine.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 25.5min finished


Fine search best RMSE: 0.9043991912876149
Fine search best params: {'n_factors': 25, 'lr_all': 0.005, 'reg_all': 0.07500000000000001, 'n_epochs': 30}


In [19]:
books_df = books_df.dropna(subset=['description', 'title'])

In [20]:
books_df.isnull().sum()

book_id                  0
isbn                242987
isbn13              186795
work_id                  0
language_code       247167
title                    0
authors                  0
publisher           140311
publication_year    115484
average_rating           0
description              0
image_url                0
url                      0
dtype: int64

## Base de données pour les datasets créés

In [13]:
books_authors = pd.read_csv('./reco_books/data/interim/goodreads/books_authors.csv')
ratings = pd.read_csv('./reco_books/data/processed/goodreads/collaborative_dataset.csv')

In [75]:
books_authors.head()

,book_id,isbn,isbn13,work_id,language_code,title,authors,publisher,publication_year,average_rating,description,image_url,url
0,1,0439785960,9780439785969,41335427.0,eng,Harry Potter and the Half-Blood Prince (Harry ...,"J.K. Rowling, Mary GrandPre",Scholastic Inc.,2006.0,4.0,The war against Voldemort is not going well: e...,https://images.gr-assets.com/books/1361039191m...,https://www.goodreads.com/book/show/1.Harry_Po...
1,2,0439358078,9780439358071,2809203.0,eng,Harry Potter and the Order of the Phoenix (Har...,"J.K. Rowling, Mary GrandPre",Scholastic Inc.,2004.0,4.0,Harry Potter is due to start his fifth year at...,https://images.gr-assets.com/books/1507396732m...,https://www.goodreads.com/book/show/2.Harry_Po...
2,3,0439554934,9780439554930,4640799.0,eng,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPre",Scholastic Inc,1997.0,4.0,Harry Potter's life is miserable. His parents ...,https://images.gr-assets.com/books/1474154022m...,https://www.goodreads.com/book/show/3.Harry_Po...
3,4,0439554896,9780439554893,6231171.0,eng,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,Scholastic,2003.0,4.0,The Dursleys were so mean and hideous that sum...,https://images.gr-assets.com/books/1474154604m...,https://www.goodreads.com/book/show/4.Harry_Po...
4,5,043965548X,9780439655484,2402163.0,eng,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPre",Scholastic Inc.,2004.0,4.0,Harry Potter's third year at Hogwarts is full ...,https://images.gr-assets.com/books/1499277281m...,https://www.goodreads.com/book/show/5.Harry_Po...


In [10]:
BIGINT_COLUMNS_BOOKS = ["book_id", "work_id"]
BIGINT_COLUMNS_RATINGS = ["book_id"]

In [11]:
for col in BIGINT_COLUMNS_BOOKS:
        if col in books_authors.columns:
            books_authors[col] = books_authors[col].fillna(0).astype(int)

In [4]:
user='postgres'
password='redsql'

import psycopg2

In [ ]:


conn = psycopg2.connect(
    host="localhost",
    dbname="Library",
    user=user,
    password=password
)
cur = conn.cursor()

with open("D:\TER\scripts_psql\create_library_db.sql", "r", encoding="utf-8") as f:
    sql = f.read()
    cur.execute(sql)

conn.commit()
cur.close()
conn.close()

In [ ]:


conn = psycopg2.connect(
    host="localhost",
    dbname="Library",
    user=user,
    password=password
)
cur = conn.cursor()

with open("D:\TER\scripts_psql\insert_library_data.sql", "r", encoding="utf-8") as f:
    sql = f.read()
    cur.execute(sql)

conn.commit()
cur.close()
conn.close()

In [10]:
conn = psycopg2.connect(
    host="localhost",
    dbname="Library",
    user=user,
    password=password
)
cur = conn.cursor()

with open("D:\TER\scripts_psql\create_users_table.sql", "r", encoding="utf-8") as f:
    sql = f.read()
    cur.execute(sql)

conn.commit()
cur.close()
conn.close()

In [9]:
conn = psycopg2.connect(
    host="localhost",
    dbname="Library",
    user=user,
    password=password
)
cur = conn.cursor()

with open("D:\TER\scripts_psql\create_indexes_library.sql", "r", encoding="utf-8") as f:
    sql = f.read()
    cur.execute(sql)

conn.commit()
cur.close()
conn.close()

In [4]:
user='postgres'
password='redsql'
from sqlalchemy import create_engine


def get_engine():
    return create_engine(
        f"postgresql://{user}:{password}@{'localhost'}/{'Library'}"
    )

engine = get_engine()

users = pd.read_sql("SELECT * FROM users", engine)

In [5]:
users.head()

,user_id,user_index,username
0,f892f696822ab435f03ad523d0d28269,11579,user11579
1,ef5df7f9065022d65810776875a6d0c3,11161,user11161
2,99b3c1ee79b8db72b5578df341aa1377,7203,user7203
3,0d98faf9357e0445dffece6a9a7ed211,649,user649
4,4ac1d06f681aae059d225d1820caac53,3550,user3550


In [7]:
users.sort_values(by='user_index', inplace=True)

In [8]:
users.to_csv('./reco_books/data/raw/goodreads/users.csv', index=False)